In [ ]:
import cv2
import json
import time
import threading
import logging
import argparse
import os
from cvzone.HandTrackingModule import HandDetector
from pynput.keyboard import Controller

# Configure Logging
logging.basicConfig(level=logging.WARNING)

# Handle command-line arguments safely
parser = argparse.ArgumentParser(description="Virtual Hand Keyboard")
parser.add_argument("--confidence", type=float, default=0.8, help="Hand detection confidence level")
parser.add_argument("--max_hands", type=int, default=2, help="Max number of hands to detect")

# Prevent errors in interactive mode (Jupyter)
try:
    args = parser.parse_args([])
except SystemExit:
    args = argparse.Namespace(confidence=0.8, max_hands=2)  # Use default values

# Check if the keyboard layout file exists, otherwise create it
if not os.path.exists("keyboard_layout.json"):
    default_keys = {
        "rows": [
            ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
            ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
            ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"],
            ["Space", "Delete"]
        ]
    }
    with open("keyboard_layout.json", "w") as f:
        json.dump(default_keys, f, indent=4)

# Load the keyboard layout from the JSON file
with open("keyboard_layout.json", "r") as f:
    keys = json.load(f)["rows"]

# Initialize Webcam
cap = cv2.VideoCapture(1)
cap.set(3, 1280)
cap.set(4, 720)
cap.set(cv2.CAP_PROP_FPS, 60)  # Optimize FPS

# Get screen width and height
screen_width = int(cap.get(3))  # 1280
screen_height = int(cap.get(4))  # 720

# Initialize Hand Detector
detector = HandDetector(detectionCon=args.confidence, maxHands=args.max_hands)

finalText = ""
keyboard = Controller()
frame_count = 0
last_press_time = {}

# Button Class
class Button:
    def __init__(self, pos, text, size):
        self.pos = pos
        self.size = size
        self.text = text

# Calculate button size dynamically based on screen size
button_width = screen_width // 12
button_height = screen_height // 12
spacing_x = 15
spacing_y = 20

# Create Buttons
buttonList = []
for i, row in enumerate(keys):
    for j, key in enumerate(row):
        if key == "Space":
            space_width = button_width * 5  # Make it wide
            buttonList.append(Button([screen_width // 2 - space_width // 2, i * (button_height + spacing_y) + 100], key, [space_width, button_height]))
        elif key == "Delete":
            delete_width = button_width * 2
            buttonList.append(Button([screen_width - delete_width - 50, i * (button_height + spacing_y) + 100], key, [delete_width, button_height]))
        else:
            buttonList.append(Button([j * (button_width + spacing_x) + 50, i * (button_height + spacing_y) + 100], key, [button_width, button_height]))

# Draw Buttons
def drawAll(img, buttonList):
    for button in buttonList:
        x, y = button.pos
        w, h = button.size
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 2)
        cv2.putText(img, button.text, (x + w // 3, y + h // 2 + 5),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
    return img

# Simulate Keyboard Press in a Separate Thread
def press_key(key):
    keyboard.press(key)
    keyboard.release(key)

while True:
    frame_count += 1
    success, img = cap.read()
    if not success or img is None:
        logging.warning("Failed to grab frame. Check camera index.")
        break

    img = cv2.flip(img, 1)  # Flip horizontally for natural interaction

    # Process every 3rd frame to reduce CPU load
    hands, img = detector.findHands(img, draw=False) if frame_count % 3 == 0 else ([], img)

    img = drawAll(img, buttonList)
    hand_types = []

    if hands:
        for hand in hands:
            lmList = hand['lmList']
            bbox = hand['bbox']
            handType = "Right" if hand['type'] == "Left" else "Left"
            hand_types.append(handType)

            x, y, w, h = bbox
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            if len(lmList) > 12:
                for button in buttonList:
                    bx, by = button.pos
                    bw, bh = button.size

                    if bx < lmList[8][0] < bx + bw and by < lmList[8][1] < by + bh:
                        cv2.rectangle(img, (bx - 5, by - 5), (bx + bw + 5, by + bh + 5), (175, 0, 175), cv2.FILLED)
                        cv2.putText(img, button.text, (bx + 20, by + 55),
                                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
                        p1 = tuple(lmList[8][:2])  # Index finger tip
                        p2 = tuple(lmList[12][:2])  # Middle finger tip
                        l, _, _ = detector.findDistance(p1, p2, img)

                        if l < 30 and (button.text not in last_press_time or time.time() - last_press_time[button.text] > 0.3):
                            if button.text == "Space":
                                finalText += " "
                            elif button.text == "Delete":
                                if len(finalText) > 0:
                                    finalText = finalText[:-1]  # Remove last character
                            else:
                                finalText += button.text
                                threading.Thread(target=press_key, args=(button.text,)).start()
                            last_press_time[button.text] = time.time()

    # Display Instructions Overlay (FULL BLACK BACKGROUND)
    cv2.rectangle(img, (10, 10), (screen_width - 10, 80), (50, 50, 50), -1)
    cv2.putText(img, "Raise finger to select | Tap index+middle to press", 
                (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # Display Final Typed Text
    cv2.rectangle(img, (50, screen_height - 100), (screen_width - 50, screen_height - 50), (175, 0, 175), cv2.FILLED)
    cv2.putText(img, finalText, (60, screen_height - 65), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)

    # Show Frame
    cv2.imshow("Virtual Keyboard", img)
    if cv2.waitKey(1) == 27:  # Press 'ESC' to Exit
        break

cap.release()
cv2.destroyAllWindows()
